In [10]:
# Core libraries
import os
import pandas as pd

# Environment variables
from dotenv import load_dotenv, dotenv_values

# Text processing
import spacy
import trafilatura

# OpenAI
from openai import OpenAI

# Notebook display utilities
from IPython.display import display, Markdown

# Load Db2 notebook helpers (CONNECT, %sql, etc.)
if not os.path.isfile("db2.ipynb"):
    os.system("wget https://raw.githubusercontent.com/IBM/db2-jupyter/master/db2.ipynb")

%run db2.ipynb

Db2 Extensions Loaded. Version: 2024-09-16


In [2]:
# Load environment variables
load_dotenv(".env", override=True)

# Db2 credentials (used by db2.ipynb's %sql CONNECT CREDENTIALS)
db2creds = dotenv_values(".env")

# Connect to Db2
%sql CONNECT CREDENTIALS db2creds

# OpenAI setup
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY is not set in .env")

EMBED_MODEL = os.getenv("OPENAI_EMBEDDING_MODEL", "text-embedding-3-small")
CHAT_MODEL = os.getenv("OPENAI_CHAT_MODEL", "gpt-4.1-mini")

client = OpenAI()


Connection successful. test1 @ localhost 


In [3]:
def get_embedding(text: str):
    """
    Generate a vector embedding for the given text using OpenAI.
    """
    text = text.replace("\n", " ")
    resp = client.embeddings.create(
        model=EMBED_MODEL,
        input=text
    )
    return resp.data[0].embedding  # list[float]


def chat_with_context(context: str, question: str) -> str:
    """
    Call OpenAI chat model with retrieved context and question.
    """
    system_prompt = (
        "You are a knowledgeable assistant. Answer the question based solely "
        "on the provided context. If the information is not in the context, "
        "respond with: 'The information is not available in the provided context.'"
    )

    user_msg = f"Context:\n{context}\n\nQuestion:\n{question}\n\nAnswer:"

    resp = client.chat.completions.create(
        model=CHAT_MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_msg},
        ],
        temperature=0.2,
        max_tokens=512,
    )

    return resp.choices[0].message.content

In [4]:
# Example URL – you can replace this with any article / blog
url = "https://www.ibm.com/blog/ai-and-data"  # change this if you like

downloaded = trafilatura.fetch_url(url)
if not downloaded:
    raise RuntimeError(f"Failed to download content from {url}")

text = trafilatura.extract(downloaded, include_comments=False, include_tables=False)
if not text:
    raise RuntimeError("Failed to extract main content from the page")

print(text[:1000])  # peek into first 1000 chars

Learn how to put AI to work
Become an expert on fundamental and emerging tech topics with our comprehensive guides
Reader favorites from the year so far
Top analysis of cyber threats
Listen to engaging discussions with tech leaders. Watch the latest episodes.
AI Academy |
Uniting security and governance for the future of AIThis AI Academy episode explores the tug-of-war that risk and assurance leaders experience between governance and security.
Explore expert-led sessions on AI agents, data for AI, AI models, AI automation, and AI governance & security


In [5]:
import en_core_web_sm
nlp = en_core_web_sm.load()

def chunk_text(text: str, max_tokens: int = 200, overlap: int = 40):
    """
    Very simple sentence-based chunking with overlap.
    """
    doc = nlp(text)
    sentences = [sent.text.strip() for sent in doc.sents if sent.text.strip()]

    chunks = []
    current_chunk = []
    current_len = 0

    for sent in sentences:
        length = len(sent.split())
        if current_len + length > max_tokens and current_chunk:
            chunks.append(" ".join(current_chunk))
            # Overlap
            overlap_tokens = " ".join(" ".join(current_chunk).split()[-overlap:])
            current_chunk = [overlap_tokens, sent]
            current_len = len(overlap_tokens.split()) + length
        else:
            current_chunk.append(sent)
            current_len += length

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

chunks = chunk_text(text, max_tokens=200, overlap=40)
len(chunks), chunks[0][:500]

(1,
 'Learn how to put AI to work\nBecome an expert on fundamental and emerging tech topics with our comprehensive guides\nReader favorites from the year so far\nTop analysis of cyber threats\nListen to engaging discussions with tech leaders. Watch the latest episodes. AI Academy |\nUniting security and governance for the future of AIThis AI Academy episode explores the tug-of-war that risk and assurance leaders experience between governance and security. Explore expert-led sessions on AI agents, data for ')

In [6]:
# Drop and recreate embeddings table
%sql DROP TABLE IF EXISTS embeddings

SQL_CREATE_TABLE = """
CREATE TABLE embeddings (
    id INT NOT NULL GENERATED ALWAYS AS IDENTITY (START WITH 1, INCREMENT BY 1),
    content CLOB,
    embedding VECTOR(1536, FLOAT32),
    PRIMARY KEY (id)
)
"""
%sql {SQL_CREATE_TABLE}

Command completed.
Command completed.


In [7]:
values = []
for chunk in chunks:
    vec = get_embedding(chunk)
    vec_str = "[" + ",".join(str(x) for x in vec) + "]"
    values.append((chunk, vec_str))

# Prepare SQL for inserting
stmt = %sql prepare INSERT INTO embeddings (content, embedding) VALUES (?, VECTOR(?, 1536, FLOAT32))

# Disable autocommit
%sql autocommit off

for content, vec_str in values:
    %sql execute :stmt using :content, :vec_str

%sql commit work
%sql autocommit on

In [8]:
# ------------------------------------
# 1) Define the question + embedding
# ------------------------------------
question = "What areas of AI and security does this page mention?"
query_vec = get_embedding(question)  # OpenAI embedding for the question


# ------------------------------------
# 2) Create query_vec table in Db2
# ------------------------------------
%sql DROP TABLE IF EXISTS query_vec

SQL_CREATE_QUERY_VEC = """
CREATE TABLE query_vec (
    id INT,
    embedding VECTOR(1536, FLOAT32)
)
"""
%sql {SQL_CREATE_QUERY_VEC}


# ------------------------------------
# 3) Insert the query embedding
# ------------------------------------
query_vec_str = "[" + ",".join(str(x) for x in query_vec) + "]"

stmt = %sql prepare INSERT INTO query_vec (id, embedding) VALUES (1, VECTOR(?, 1536, FLOAT32))

%sql autocommit off
%sql execute :stmt using :query_vec_str
%sql commit work
%sql autocommit on


# ------------------------------------
# 4) Run vector search (CAST to VARCHAR to avoid CLI0109E)
# ------------------------------------
top_k = 5

SQL_DISTANCE = f"""
SELECT 
    e.id,
    CAST(SUBSTR(e.content, 1, 4000) AS VARCHAR(4000)) AS CONTEXT,
    VECTOR_DISTANCE(
        e.embedding,
        q.embedding,
        EUCLIDEAN
    ) AS DISTANCE
FROM embeddings e, query_vec q
ORDER BY DISTANCE ASC
FETCH FIRST {top_k} ROWS ONLY
"""

result_df = %sql {SQL_DISTANCE}
display(result_df)

Command completed.
Command completed.


,ID,CONTEXT,DISTANCE
0,1,Learn how to put AI to work\nBecome an expert ...,0.920621


In [9]:
# Db2 returns column names in uppercase unless quoted
context_column = "CONTEXT" if "CONTEXT" in result_df.columns else "context"

# Build a merged context string
context_str = "\n\n---\n\n".join(result_df[context_column].tolist())

# Ask LLM with context (RAG)
answer = chat_with_context(context_str, question)

display(Markdown(f"**Q:** {question}\n\n**A:** {answer}"))

**Q:** What areas of AI and security does this page mention?

**A:** The page mentions AI agents, data for AI, AI models, AI automation, and AI governance & security as areas of AI and security.